# OpenBeer - DataLoader - Notebook  
  
### 0. Load modules

In [2]:
#Load modules and set seaborn styles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()

### I. Read Data

In [5]:
#set columns from header row of csv file and path
columns_to_load = ["id","brewery_id","name","cat_id","style_id","abv","ibu","srm","upc","filepath","descript","last_mod"]
path = "openbeerdb_csv\\beers.csv"

#read raw data
df_raw = pd.read_csv(path, index_col="id", usecols=columns_to_load)

#explore top 5 rows if loaded correctly
print(df_raw.head())


comment ="""
Column explanation:
id = ID of the entry
brewery_id = ID of the corresponding brewery
name = Name of the beer
cat_id = ID of the beer category
style_id = ID of the beer style
abv = ??
ibu = ??
srm = ??
filepath = ??
descript = Description of the beer style
last_mod = Date/Time of last modification of the entry
"""

brewery_id                    name cat_id style_id  abv ibu srm upc  \
id                                                                       
1         812             Hocus Pocus     11      116  4.5   0   0   0   
2         264       Grimbergen Blonde     -1       -1  6.7   0   0   0   
3         779  Widdershins Barleywine     -1       -1  9.1   0   0   0   
4         287                 Lucifer     -1       -1  8.5   0   0   0   
5        1056                  Bitter     -1       -1    4   0   0   0   

   filepath                                           descript  \
id                                                               
1       NaN  Our take on a classic summer ale.  A toast to ...   
2       NaN                                                NaN   
3       NaN                                                NaN   
4       NaN                                                NaN   
5       NaN                                                NaN   

               last_m